## Description

I wants understand why my train is different on other people where i wrong?
I train default how i train another model detection and convolution but i see in kernels in kaggle people use another techicks to train nlp model and i can't understand why and i want test it.

Links to use:
- [22nd solution](https://www.kaggle.com/c/commonlitreadabilityprize/discussion/257302)

- [gtihub link](https://github.com/kurupical/commonlit)

- [RoBERTa Base Fine-Tuning with Better Training Strategies](https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-fit)

## default or not changed loads

In [ ]:
from IPython.display import clear_output
!pip install transformers
clear_output()

In [ ]:
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import  DataLoader, Dataset
import transformers

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'


SEED =13
np.random.seed(SEED)
torch.manual_seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def make_folds(data: pd.DataFrame, split: int = 5):
    data['kfold'] = -1
    data = data.sample(frac =1).reset_index(drop=True)
    num_bins = int(np.floor(1 + np.log2(len(data))))
    data.loc[:, 'bins'] = pd.cut(
        data['target'], bins = num_bins, labels = False
    )
    kf = model_selection.StratifiedKFold(n_splits=split)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    data = data.drop("bins", axis=1)
    return data


def clean_text(txt):
    return re.sub('[^A-Za-z]+', ' ',str(txt).lower())


df = pd.read_csv(path_tr)
df['txt'] = df['excerpt']#.apply(lambda x: clean_text(x))
df_folds = make_folds(df, 5)

In [ ]:
class CL_Dataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        token,
        max_len: int = 256,
        test: bool = False
        ) -> dict:
        self.data = data 
        self.max_len = max_len
        self.test = test
        self.token = token

    def __len__(self):
        return self.data.shape[0]

    def  __getitem__(self, idx: int):
        text = self.data.txt.iloc[idx]
        encode = self.token(
            text,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            add_special_tokens=True,            
            return_attention_mask=True,
            return_token_type_ids=False  
            )
        if self.test:
            target = 0
        else:
            target = self.data.target.iloc[idx]

        ids = encode["input_ids"]
        mask = encode["attention_mask"]

        return {
            "input_ids": torch.tensor(ids, dtype=torch.long),
            "attention_mask": torch.tensor(mask, dtype=torch.long),
            'target': torch.tensor(target, dtype = torch.float)  
        }

In [ ]:
def loss_fn(output,target):
    return torch.sqrt(nn.MSELoss()(output,target))


def viz_curve(data: dict, title: str) -> None:
    epochs = list(range(1, EPOCH + 1))
    fig = make_subplots(
        rows=2, cols=2,
        specs=[
               [{"colspan": 2}, None],
               [{}, {}],
               ],
               vertical_spacing=0.09,
               subplot_titles=('Loss Curve',  'RMSE', 'LR')
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['train_loss'],
            mode='lines+markers',
            name='Train Loss'),
            row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['valid_loss'],
            mode='lines+markers',
            name='Valid Loss'),
            row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['valid_rms'],
            mode='lines+markers',
            name='RMSE'),     
            row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['lr'],
            mode='lines+markers',
            name='LR'), 
            row=2, col=2
    )
    fig.update_layout(
        height=600,
        width=600,
        showlegend=False,
        title = title,
        margin=dict(l=10, r=10, t=30, b=20),                     
        template="plotly_dark"    
    )
    fig.show()

def update_result(
    data: dict,
    predict: torch.tensor,
    target:torch.tensor
) -> None:
    loss = loss_fn(predict.squeeze(-1), target)
    data['losses'].append(loss.detach().cpu().numpy())
    data['all_pred'].append(predict.squeeze(-1).detach().cpu().numpy())
    data['all_target'].append(target.detach().cpu().numpy())
    return loss


def train(
    model:nn.Module, 
    loader: DataLoader,
    optimizer: transformers.AdamW,
    schedule: transformers.get_linear_schedule_with_warmup,
    batch: int,
    max_lenght: int
) -> list:
    model.train()
    torch.backends.cudnn.benchmark = True
    results = defaultdict(list)
    for input in loader:
        optimizer.zero_grad()
        target = input['target'].to(device)
        batch = {k:v.to(device) for k,v in input.items() if k != 'target'}   
        out = model(**batch)
        out = out.logits # BertForSequenceClassification
        loss = update_result(results, out, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()    
        schedule.step()
    losses = np.mean(results['losses'])
    allt = np.concatenate(results['all_target'])
    allp = np.concatenate(results['all_pred'])
    rmse = np.sqrt(mean_squared_error(allt, allp))  
    return losses, rmse


def validate(
    model:nn.Module, 
    loader: DataLoader, 
    batch: int,
    max_lenght: int
) -> list:
    model.eval()
    results = defaultdict(list)
    for input in loader:  
        target = input['target'].to(device)
        batch = {k:v.to(device) for k,v in input.items() if k != 'target'} 
        out = model(**batch)
        out = out.logits # BertForSequenceClassification
        _ = update_result(results, out, target)
    losses = np.mean(results['losses'])
    allt = np.concatenate(results['all_target'])
    allp = np.concatenate(results['all_pred'])
    rmse = np.sqrt(mean_squared_error(allt, allp)) 
    return losses, rmse


def showtime(model:nn.Module, data: pd.DataFrame, tokenizer:transformers.AutoTokenizer, fold: int) -> dict:    
    history = defaultdict(list)
    model = model.to(device)
    ttr = data[data.kfold != fold].reset_index(drop=True)
    vvl = data[data.kfold == fold].reset_index(drop=True)
    print(f'Fold: {fold + 1}, --- {ttr.shape, vvl.shape}')

    tr = CL_Dataset(ttr,tokenizer, MAX_LEN)
    vl = CL_Dataset(vvl,tokenizer, MAX_LEN)
    tr_loader = DataLoader(
        tr,
        batch_size=BATCH,
        shuffle=True
    )
    vl_loader = DataLoader(
        vl,
        batch_size=BATCH,
        shuffle=False
    )
    optimizer = transformers.AdamW(model.parameters(), lr=2e-5, correct_bias=True)
    steps = len(tr_loader) * EPOCH
    print(steps, len(tr_loader))
    # steps = len(tr_loader)/BATCH * EPOCH
    lin_schedule = transformers.get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=1,
        num_training_steps=steps
    )
    best_rmse = np.inf
    for epoch in tqdm(range(EPOCH)):
        tr_loss, tr_rmse = train(model, tr_loader, optimizer, lin_schedule, BATCH, MAX_LEN)
        vl_loss, vl_rmse = validate(model, vl_loader, BATCH, MAX_LEN)
        lr = optimizer.param_groups[0]['lr']
        history['train_loss'].append(tr_loss)
        history['valid_loss'].append(vl_loss)
        history['valid_rms'].append(vl_rmse)
        history['lr'].append(lr)
        print(f'Epoch: {epoch}, lr: {lr}, train rmse: {tr_rmse}, vl rmse: {vl_rmse}, vl loss: {vl_loss}')
        if vl_rmse < best_rmse:
            print(f'Save rmse: {vl_rmse}')
            torch.save(model.state_dict(), f'{MODEL}_model_{fold}.pth')
            best_rmse = vl_rmse
    return history

In [ ]:
class Auto_NLP_model:
    def __init__(self, name: str, n_labels: int = 1):
        print(f'Auto NLP init model : {name}')
        self.name = name
        self.n_labels=  n_labels        
    
    def loads(self):
        print('Loading configuraiton...')
        model_config = transformers.AutoConfig.from_pretrained(
                pretrained_model_name_or_path=self.name,
                num_labels=self.n_labels)
        print('Loading tokenizer...')
        tokenizer = transformers.AutoTokenizer.from_pretrained(
                pretrained_model_name_or_path=self.name)
        print('Loading model...')
        model = transformers.AutoModelForSequenceClassification.from_pretrained(
                pretrained_model_name_or_path=self.name,
                config=model_config)
        return model, tokenizer

In [ ]:
"""
https://huggingface.co/transformers/pretrained_models.html
https://huggingface.co/transformers/model_doc/auto.html

"""

m = {
    # 'xlm-roberta':'xlm-roberta-base',
    # 'deberta': 'microsoft/deberta-base',
    # 'xlm':'xlm-mlm-ende-1024',
    # 'camembert': 'camembert-base',
    # 'mbart': 'facebook/mbart-large-cc25', #memory batch 4 not ends error
    'roberta': 'roberta-base' #roberta-large'
}

In [ ]:
EPOCH = 15
BATCH = 6
MAX_LEN = 256

for k, v in m.items():
    print(k)
    model, tokenizer = Auto_NLP_model(v).loads()
    MODEL = model.__class__.__name__
    fold = 0
    title = f'Model: {k} Fold: {fold + 1}'
    history = showtime(model, df_folds, tokenizer, fold)
    viz_curve(history, title)

roberta
Auto NLP init model : roberta-base
Loading configuraiton...
Loading tokenizer...
Loading model...


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Fold: 1, --- ((2267, 8), (567, 8))
5670 378


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 0, lr: 1.866995942847063e-05, train rmse: 0.7436621189117432, vl rmse: 0.6540188789367676, vl loss: 0.6269398927688599
Save rmse: 0.6540188789367676


...

Epoch: 12, lr: 2.66713706121009e-06, train rmse: 0.14275813102722168, vl rmse: 0.585809588432312, vl loss: 0.562573254108429


 93%|█████████▎| 14/15 [56:48<04:04, 244.24s/it]

Epoch: 13, lr: 1.333568530605045e-06, train rmse: 0.13657480478286743, vl rmse: 0.5636139512062073, vl loss: 0.5399689674377441


100%|██████████| 15/15 [1:00:53<00:00, 243.57s/it]

Epoch: 14, lr: 0.0, train rmse: 0.12706246972084045, vl rmse: 0.5606884360313416, vl loss: 0.5367791652679443


### results:

results model roberta-large(~1h:45min):

```
# This is formatted as code
Fold: 1, --- ((2267, 8), (567, 8))
  0%|          | 0/15 [00:00<?, ?it/s]Epoch: 0, lr: 1.866995942847063e-05, train rmse: 0.771242618560791, vl rmse: 0.5320066809654236, vl loss: 0.5065193772315979
Save rmse: 0.5320066809654236
 13%|█▎        | 2/15 [13:49<1:30:08, 416.03s/it]Epoch: 1, lr: 1.7336390897865588e-05, train rmse: 0.5480484366416931, vl rmse: 0.7261704802513123, vl loss: 0.7056165933609009
 20%|██        | 3/15 [20:51<1:23:43, 418.63s/it]Epoch: 2, lr: 1.6002822367260542e-05, train rmse: 0.45481783151626587, vl rmse: 0.5933656096458435, vl loss: 0.5674742460250854
 27%|██▋       | 4/15 [27:52<1:16:56, 419.67s/it]Epoch: 3, lr: 1.4669253836655498e-05, train rmse: 0.3679790794849396, vl rmse: 0.6310817003250122, vl loss: 0.6084640622138977
 33%|███▎      | 5/15 [34:54<1:10:04, 420.41s/it]Epoch: 4, lr: 1.333568530605045e-05, train rmse: 0.3390931189060211, vl rmse: 0.7398288249969482, vl loss: 0.721051037311554
 40%|████      | 6/15 [41:56<1:03:08, 420.91s/it]Epoch: 5, lr: 1.2002116775445406e-05, train rmse: 0.2956746220588684, vl rmse: 0.5392723083496094, vl loss: 0.5154401659965515
 47%|████▋     | 7/15 [48:58<56:11, 421.40s/it]  Epoch: 6, lr: 1.066854824484036e-05, train rmse: 0.2643064260482788, vl rmse: 0.5453745126724243, vl loss: 0.5227868556976318
 53%|█████▎    | 8/15 [56:01<49:11, 421.65s/it]Epoch: 7, lr: 9.334979714235315e-06, train rmse: 0.25952309370040894, vl rmse: 0.6315166354179382, vl loss: 0.6088392734527588
 60%|██████    | 9/15 [1:03:03<42:11, 421.85s/it]Epoch: 8, lr: 8.001411183630271e-06, train rmse: 0.2210676074028015, vl rmse: 0.659906804561615, vl loss: 0.6411230564117432
 67%|██████▋   | 10/15 [1:10:05<35:10, 422.07s/it]Epoch: 9, lr: 6.667842653025225e-06, train rmse: 0.21520286798477173, vl rmse: 0.5818750262260437, vl loss: 0.5584777593612671
Epoch: 10, lr: 5.33427412242018e-06, train rmse: 0.20328344404697418, vl rmse: 0.5239043235778809, vl loss: 0.4989898204803467
Save rmse: 0.5239043235778809
 80%|████████  | 12/15 [1:24:15<21:09, 423.21s/it]Epoch: 11, lr: 4.0007055918151355e-06, train rmse: 0.18055203557014465, vl rmse: 0.5367754697799683, vl loss: 0.5130714178085327
 87%|████████▋ | 13/15 [1:31:17<14:05, 422.95s/it]Epoch: 12, lr: 2.66713706121009e-06, train rmse: 0.1665629744529724, vl rmse: 0.5970629453659058, vl loss: 0.5752689838409424
 93%|█████████▎| 14/15 [1:38:19<07:02, 422.65s/it]Epoch: 13, lr: 1.333568530605045e-06, train rmse: 0.15745727717876434, vl rmse: 0.5785849094390869, vl loss: 0.5563144683837891
100%|██████████| 15/15 [1:45:21<00:00, 421.43s/it]Epoch: 14, lr: 0.0, train rmse: 0.14649353921413422, vl rmse: 0.5729362964630127, vl loss: 0.5512558817863464

make small version robrta base 1h:

Fold: 1, --- ((2267, 8), (567, 8))
5670 378
  0%|          | 0/15 [00:00<?, ?it/s]Epoch: 0, lr: 1.866995942847063e-05, train rmse: 0.7436621189117432, vl rmse: 0.6540188789367676, vl loss: 0.6269398927688599
Save rmse: 0.6540188789367676
  7%|▋         | 1/15 [04:03<56:50, 243.59s/it]Epoch: 1, lr: 1.7336390897865588e-05, train rmse: 0.5260870456695557, vl rmse: 0.6349746584892273, vl loss: 0.6099616289138794
Save rmse: 0.6349746584892273
 20%|██        | 3/15 [12:09<48:35, 242.93s/it]Epoch: 2, lr: 1.6002822367260542e-05, train rmse: 0.4299793243408203, vl rmse: 0.6682462096214294, vl loss: 0.6423256397247314
Epoch: 3, lr: 1.4669253836655498e-05, train rmse: 0.36037909984588623, vl rmse: 0.543418824672699, vl loss: 0.5201668739318848
Save rmse: 0.543418824672699
 33%|███▎      | 5/15 [20:15<40:29, 242.91s/it]Epoch: 4, lr: 1.333568530605045e-05, train rmse: 0.3064141869544983, vl rmse: 0.5811491012573242, vl loss: 0.5575568079948425
 40%|████      | 6/15 [24:17<36:24, 242.74s/it]Epoch: 5, lr: 1.2002116775445406e-05, train rmse: 0.2750021517276764, vl rmse: 0.582738995552063, vl loss: 0.5591065883636475
Epoch: 6, lr: 1.066854824484036e-05, train rmse: 0.241420716047287, vl rmse: 0.5221829414367676, vl loss: 0.4948228895664215
Save rmse: 0.5221829414367676
 53%|█████▎    | 8/15 [32:23<28:20, 242.88s/it]Epoch: 7, lr: 9.334979714235315e-06, train rmse: 0.22280079126358032, vl rmse: 0.5436638593673706, vl loss: 0.5197162628173828
 60%|██████    | 9/15 [36:26<24:16, 242.72s/it]Epoch: 8, lr: 8.001411183630271e-06, train rmse: 0.1977776437997818, vl rmse: 0.5721029043197632, vl loss: 0.5503267645835876
 67%|██████▋   | 10/15 [40:30<20:16, 243.21s/it]Epoch: 9, lr: 6.667842653025225e-06, train rmse: 0.18672426044940948, vl rmse: 0.5271970629692078, vl loss: 0.5030388236045837
 73%|███████▎  | 11/15 [44:35<16:14, 243.60s/it]Epoch: 10, lr: 5.33427412242018e-06, train rmse: 0.16900299489498138, vl rmse: 0.5574540495872498, vl loss: 0.5334554314613342
 80%|████████  | 12/15 [48:39<12:11, 243.90s/it]Epoch: 11, lr: 4.0007055918151355e-06, train rmse: 0.15365661680698395, vl rmse: 0.5263659954071045, vl loss: 0.5023990273475647
 87%|████████▋ | 13/15 [52:44<08:08, 244.14s/it]Epoch: 12, lr: 2.66713706121009e-06, train rmse: 0.14275813102722168, vl rmse: 0.585809588432312, vl loss: 0.562573254108429
 93%|█████████▎| 14/15 [56:48<04:04, 244.24s/it]Epoch: 13, lr: 1.333568530605045e-06, train rmse: 0.13657480478286743, vl rmse: 0.5636139512062073, vl loss: 0.5399689674377441
100%|██████████| 15/15 [1:00:53<00:00, 243.57s/it]Epoch: 14, lr: 0.0, train rmse: 0.12706246972084045, vl rmse: 0.5606884360313416, vl loss: 0.5367791652679443


```

## adds something new
readme:
- https://huggingface.co/transformers/model_summary.html
 

TODO:
- https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-itpt
- https://huggingface.co/blog/accelerate-library

PS:
- https://www.kaggle.com/c/commonlitreadabilityprize/discussion/257629



### [huggingface course](https://huggingface.co/course/chapter1)

«Повторение — мать учения и прибежище ослов»

In [1]:
from IPython.display import clear_output, Image
!pip install transformers
clear_output()

In [2]:
import transformers

#### pipelines

```
Some of the currently available pipelines are:
- feature-extraction (get the vector representation of a text)
- fill-mask
- ner (named entity recognition)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification
```

Zero-shot classification
Мы начнем с решения более сложной задачи, когда нам нужно классифицировать тексты без маркировки. 

Это распространенный сценарий в реальных проектах, потому что аннотирование текста обычно занимает много времени и требует знаний в предметной области.

Для этого варианта использования конвейер классификации с нулевым выстрелом очень эффективен: **он позволяет указать, какие метки использовать для классификации, поэтому вам не нужно полагаться на метки предварительно обученной модели**.

Hit:
Этот конвейер называется нулевым выстрелом, потому что вам не нужно настраивать модель на ваших данных, чтобы использовать ее. Он может напрямую возвращать оценки вероятности для любого списка ярлыков, который вы хотите!

In [ ]:
clf = transformers.pipeline('zero-shot-classification')
clf('We’ll start by tackling a more challenging task where we need to classify texts that haven’t been labelled.',
    candidate_labels = ["education", "politics", "business"])

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'labels': ['education', 'business', 'politics'],
 'scores': [0.4677025079727173, 0.34642189741134644, 0.18587562441825867],
 'sequence': 'We’ll start by tackling a more challenging task where we need to classify texts that haven’t been labelled.'}

In [ ]:
clf('Alzheimer\'s disease (AD) is a neurodegenerative disease that usually starts slowly and progressively worsens.',
    candidate_labels = ['education', "neurology", 'medicine', 'business'])

{'labels': ['neurology', 'medicine', 'education', 'business'],
 'scores': [0.9273449778556824,
  0.03946782648563385,
  0.018152104690670967,
  0.015035135671496391],
 'sequence': "Alzheimer's disease (AD) is a neurodegenerative disease that usually starts slowly and progressively worsens."}

#### Hub in a pipeline
[HUB](https://huggingface.co/models)

In [ ]:
hub_model = 'xlm-roberta-base'
task = 'fill-mask'
txt = 'Go to the Model <mask> and click on the corresponding tag on the left'\
      'to display only the supported model for that task. You should get'\
      'to a page like this one.'
label = 'hub'

clf = transformers.pipeline(
    task=task,
    model=hub_model
)
# top_k определяет, сколько возможностей вы хотите отобразить.
clf(txt, top_k = 2)

[{'score': 0.18096968531608582,
  'sequence': 'Go to the Model tab and click on the corresponding tag on the leftto display only the supported model for that task. You should getto a page like this one.',
  'token': 20928,
  'token_str': 'tab'},
 {'score': 0.15255053341388702,
  'sequence': 'Go to the Model page and click on the corresponding tag on the leftto display only the supported model for that task. You should getto a page like this one.',
  'token': 9191,
  'token_str': 'page'}]

In [ ]:
hub_model = 'xlm-roberta-base'
task = 'fill-mask'
txt = 'We have model hub. Go to the Model <mask> and click on the corresponding tag on the left'\
      'to display only the supported model for that task. You should get'\
      'to a page like this one.'
label = 'hub'

clf = transformers.pipeline(
    task=task,
    model=hub_model
)
# top_k определяет, сколько возможностей вы хотите отобразить.
clf(txt, top_k = 2)

[{'score': 0.1557518094778061,
  'sequence': 'We have model hub. Go to the Model hub and click on the corresponding tag on the leftto display only the supported model for that task. You should getto a page like this one.',
  'token': 26708,
  'token_str': 'hub'},
 {'score': 0.1547161042690277,
  'sequence': 'We have model hub. Go to the Model Hub and click on the corresponding tag on the leftto display only the supported model for that task. You should getto a page like this one.',
  'token': 53465,
  'token_str': 'Hub'}]

In [ ]:
hub_model = 'dslim/bert-base-NER'
task = 'ner'
txt = 'we walked along Pushkinskaya street it was early Thursday, sun bunnies'\
      'were playing on the face of my companion Anna Degtyareva, all this was'\
      'quite recently in Berlin.'\

"""
aggregation_strategy:
    Например, Sylvain разделен на четыре части: S, ## yl, ## va и ## in.
    На этапе постобработки конвейер успешно перегруппировал эти части.

aggregation_strategy (: obj: `str`,` optional`, по умолчанию: obj: `" none "`): 
                    стратегия объединения (или нет) токенов на основе прогноза модели.
- «none»: просто не будет выполнять агрегацию и просто вернет необработанные результаты из модели.
- «simple»: будет пытаться сгруппировать объекты, следуя схеме по умолчанию. (A, B-TAG), (B, I-TAG), (C,
    I-TAG), (D, B-TAG2) (E, B-TAG2) в конечном итоге будет [{"word": ABC, "entity": "TAG"}, {"word": "D",
    "entity": "TAG2"}, {"word": "E", "entity": "TAG2"}] Обратите внимание, что два последовательных тега B будут
    разные сущности. В словесных языках мы можем в конечном итоге нежелательно разбивать слова: представьте себе
    Microsoft помечается как [{"word": "Micro", "entity": "ENTERPRISE"}, {"word": "soft», "entity":
    "НАЗВАНИЕ"}]. Ищите ПЕРВЫЙ, МАКСИМАЛЬНЫЙ, СРЕДНИЙ способы смягчения этого и устранения неоднозначности слов (на языках
    которые поддерживают это значение, которое в основном состоит из токенов, разделенных пробелом). Эти меры будут
    работать только с настоящими словами, «Нью-Йорк» все еще может быть помечен двумя разными объектами.
- "first": (работает только с моделями на основе слов) Будет использоваться стратегия: obj: `SIMPLE`, за исключением слов,
    не может заканчиваться разными тегами. Слова будут просто использовать тег первого токена слова, когда
    есть двусмысленность.
- "avarage": (работает только с моделями на основе слов) Будет использоваться стратегия: obj: `SIMPLE`, за исключением слов,
    не может заканчиваться разными тегами. баллы будут сначала усреднены по токенам, а затем максимальное
    этикетка прилагается.
- "max": (работает только с моделями на основе слов) Будет использоваться стратегия: obj: `SIMPLE`, за исключением слов,
    не может заканчиваться разными тегами. Сущность Word будет просто токеном с максимальной оценкой.

"""
clf = transformers.pipeline(
    task=task,
    model=hub_model,
    aggregation_strategy='simple'
)
clf(txt)

[{'end': 28,
  'entity_group': 'LOC',
  'score': 0.796749,
  'start': 16,
  'word': 'Pushkinskaya'},
 {'end': 126,
  'entity_group': 'PER',
  'score': 0.961868,
  'start': 111,
  'word': 'Anna Degtyareva'},
 {'end': 164,
  'entity_group': 'LOC',
  'score': 0.99955106,
  'start': 158,
  'word': 'Berlin'}]

#### Transformer models grouped into three categories:

- GPT-like (also called auto-regressive Transformer models)
- BERT-like (also called auto-encoding Transformer models)
- BART/T5-like (also called sequence-to-sequence Transformer models)

Все упомянутые выше модели Transformer (GPT, BERT, BART, T5 и т. Д.) Были обучены как языковые модели. Это означает, что они были обучены работе с большими объемами необработанного текста в режиме самоконтроля. **Самостоятельное обучение - это тип обучения, при котором цель автоматически вычисляется на основе входных данных модели. Это означает, что люди не нужны для маркировки данных!**

Этот тип модели развивает статистическое понимание языка, на котором ее обучали, но он не очень полезен для конкретных практических задач

Из-за этого общая предварительно обученная модель затем проходит процесс, называемый**трансферным обучением**.

Во время этого процесса модель настраивается контролируемым образом - то есть с использованием меток, аннотированных людьми - для данной задачи.

#### Transfer Learning

Предварительное обучение - это процесс обучения модели с нуля: веса инициализируются случайным образом, и обучение начинается без каких-либо предварительных знаний.

Предварительное обучение языковой модели требует больших затрат времени и денег.
Это предварительное обучение обычно выполняется на очень больших объемах данных. Следовательно, для этого требуется очень большой массив данных, а обучение может занять до нескольких недель.

С другой стороны, тонкая настройка - это обучение, выполняемое после предварительного обучения модели. Чтобы выполнить точную настройку, вы сначала приобретаете предварительно обученную языковую модель, а затем выполняете дополнительное обучение с набором данных, специфичным для вашей задачи. 

Предварительно обученная модель уже была обучена на наборе данных, который имеет некоторое сходство с набором данных точной настройки. Таким образом, процесс тонкой настройки может использовать знания, полученные исходной моделью во время предварительного обучения (например, с проблемами НЛП предварительно обученная модель будет иметь какое-то статистическое понимание языка, который вы используете для своей задачи).
Поскольку предварительно обученная модель уже была обучена на большом количестве данных, для точной настройки требуется гораздо меньше данных, чтобы получить достойные результаты.

Этот процесс также приведет к лучшим результатам, чем обучение с нуля (если у вас нет большого количества данных), поэтому вы всегда должны пытаться использовать предварительно обученную модель - такую, которая максимально приближена к вашей задаче, - и точно настраивать Это.

#### Encoder models

ALBERT, BERT, DistilBERT, ELECTRA, RoBERTa

На каждом этапе уровни внимания могут получить доступ ко всем словам в начальном предложении. Эти модели часто характеризуются как имеющие «двунаправленное» внимание и часто называются моделями автокодирования.

Модели кодировщика лучше всего подходят для задач, требующих понимания всего предложения, таких как классификация предложений, распознавание именованных сущностей (и, в более общем смысле, классификация слов) и извлечение ответов на вопросы.

####Decoders models 

В моделях декодеров используется только декодер модели Transformer. На каждом этапе для данного слова уровни внимания могут получить доступ только к словам, расположенным перед ним в предложении. Эти модели часто называют авторегрессивными моделями.

Предварительное обучение моделей декодеров обычно вращается вокруг предсказания следующего слова в предложении.

Эти модели лучше всего подходят для задач, связанных с генерацией текста.

К представителям этого семейства моделей относятся:

CTRL,GPT,GPT-2,Transformer XL

#### Encoder-decoder models (also called sequence-to-sequence models)

На каждом этапе уровни внимания кодера могут получить доступ ко всем словам в начальном предложении, тогда как уровни внимания декодера могут получить доступ только к словам, расположенным перед данным словом во входных данных.

Предварительное обучение этих моделей может быть выполнено с использованием целей моделей кодировщика или декодера, но обычно включает в себя что-то более сложное. Например, T5 предварительно обучается путем замены случайных отрезков текста (которые могут содержать несколько слов) одним специальным словом с маской, а затем цель состоит в том, чтобы предсказать текст, который заменяет это слово маски.

Последовательные модели лучше всего подходят для задач, связанных с генерацией новых предложений в зависимости от заданных входных данных, таких как обобщение, перевод или генеративный ответ на вопрос.

К представителям этого семейства моделей относятся:

BART, mBART, Marian, T5

#### Bias and limitations(Предвзятость и ограничения)

Если вы намереваетесь использовать предварительно обученную модель или доработанную версию в производственной среде, имейте в виду, что, хотя эти модели являются мощными инструментами, они имеют ограничения. Самая большая из них заключается в том, что для обеспечения возможности предварительного обучения на больших объемах данных исследователи часто очищают весь контент, который они могут найти, беря как лучшее, так и худшее из того, что доступно в Интернете.

На просьбу заполнить пропущенное слово в этих двух предложениях модель дает только один ответ без учета пола (официант / официантка). 
Остальные - это рабочие профессии, обычно связанные с одним конкретным полом - и да, проститутка попала в пятерку лучших возможностей, которые модель ассоциирует с понятиями «женщина» и «работа». Это происходит, несмотря на то, что BERT - одна из редких моделей Transformer, построенная не на основе сбора данных со всего Интернета, а с использованием явно нейтральных данных (она обучена на наборах данных английской Википедии и BookCorpus).

Поэтому, используя эти инструменты, вы должны помнить, что исходная модель, которую вы используете, может очень легко создать сексистский, расистский или гомофобный контент. Точная настройка модели на ваших данных не устранит эту внутреннюю предвзятость.

#### Model heads

- Model (retrieve the hidden states)
- ForCausalLM
- ForMaskedLM
- ForMultipleChoice
- ForQuestionAnswering
- ForSequenceClassification
- ForTokenClassification

Example:  So, we won’t actually use the AutoModel class for classification, but AutoModelForSequenceClassification

In [5]:
Image(url='https://huggingface.co/course/static/chapter2/transformer_and_head.png', width=800)

#### Tokenizers

1. Word-based
- Первый тип токенизатора, который приходит на ум, основан на словах или пунктуации.
2. Character-based
- Символьные токенизаторы разбивают текст на символы, а не на слова. Но и здесь возникают вопросы относительно пробелов и знаков препинания.
3. Subword tokenization
- Алгоритмы разметки подслов основаны на принципе, согласно которому часто используемые слова не следует разбивать на более мелкие подслова, а редкие слова следует разбивать на значимые подслова.
 “annoyingly” >>> “annoying” and “ly”. 

In [12]:
token = transformers.AutoTokenizer.from_pretrained('xlm-roberta-base')
txt = ['Google Research India contributes fundamental advances',
       'in computer science and applies their research to big problems impacting India',
       'Google, and communities around the world']
d = token(txt)

In [ ]:
model = transformers.AutoModel.from_pretrained('xlm-roberta-base')
model(d)

##### first 'error size'

In [ ]:
d = token(txt, return_tensors='pt')
model(d)

##### Second error : 'Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.'

Problem:
```
{'input_ids': [
    [0, 1815, 42477, 5596, 162466, 7, 20531, 129745, 7, 2],
    [0, 23, 13909, 41664, 136, 4027, 25720, 2363, 25188, 47, 6957, 44402, 24725, 214, 5596, 2], 
    [0, 1815, 4, 136, 142080, 10932, 70, 8999, 2]
    ],
'attention_mask': [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
    1, 1, 1, 1, 1, 1, 1, 1, 1]
    ]
}
```
Two way:
 - padding (заполнение)
 - truncation (усичение)

 add padding:
 ```
 {
'input_ids': tensor([
     [0, 1815,  42477,   5596, 162466,      7,  20531, 129745, 7, 2, 1, 1, 1, 1, 1, 1],
     [0, 23,  13909,  41664, 136, 4027, 25720, 2363,  25188, 47, 6957, 44402, 24725, 214, 5596, 2],
     [0, 1815, 4, 136, 142080, 10932, 70, 8999, 2,
1, 1, 1, 1, 1, 1, 1]
    ]
   ),
'attention_mask': tensor([
     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
     [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
     ]
    )
}
```
add trucation,max length = 5
```
{
'input_ids': tensor([
    [0, 1815, 42477,  5596, 2],
    [0, 23, 13909, 41664, 2],
    [0, 1815, 4, 136, 2]
    ]
  ),
'attention_mask': tensor([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1]
    ]
  )
}
```
Most models handle sequences of up to 512 or 1024 tokens

In [25]:
d = token(txt, return_tensors='pt', truncation = True, padding=True,  max_length=15)
out = model(**d)

In [26]:
out['last_hidden_state'].shape, out['pooler_output'].shape

(torch.Size([3, 15, 768]), torch.Size([3, 768]))

In [ ]:
stop

NameError: ignored

In [ ]:
from typing import Any

In [ ]:
@dataclasses.dataclass
class Config:
    experiment_name: str
    debug: bool = False
    fold: int = 0

    nlp_model_name: str = "roberta-large"
    linear_dim: int = 128
    # dropout: float = 0
    # dropout_stack: float = 0.2
    batch_size: int = 6

    lr_bert: float = 1e-5
    lr_fc: float = 1e-4
    scheduler_params = {"num_warmup_steps": batch_size*100,
                        "num_training_steps": batch_size*3000}
    if debug:
        epochs: int = 2
    else:
        epochs: int = 15

    optimizer: Any = AdamW
    weight_decay = 0.1

In [ ]:
def configure_optimizers(self):
    optimizer =  transformers.AdamW(
        params=[{"params": self.bert.parameters(), "lr": config.lr_bert},
                {"params": self.linear.parameters(), "lr": config.lr_fc}],
        weight_decay=weight_decay
    )
    scheduler = transformers.get_linear_schedule_with_warmup(optimizer, **self.config.scheduler_params)
    return [optimizer], [scheduler]

In [ ]:
model, tokenizer = Auto_NLP_model(v).loads()

first part we see lr is differen for last layer and model

```
model.roberta all model without classifier
model.classifier
>>  (classifier):
    RobertaClassificationHead(
        (dense): Linear(in_features=1024, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (out_proj): Linear(in_features=1024, out_features=1, bias=True)

-----
Fold: 1, --- ((2267, 8), (567, 8))
tr_loader len = 378 how get >> len data = 2267 / batch = 6
```

In [ ]:
2267/6

In [ ]:
list(model.parameters())

In [ ]:
model.classifier

In [ ]:
steps = len(tr_loader) * EPOCH
optimizer = transformers.AdamW(model.parameters(), lr=2e-5, correct_bias=True)
steps = len(tr_loader) * EPOCH
# steps = len(tr_loader)/BATCH * EPOCH
lin_schedule = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=1,
    num_training_steps=steps
)